# PSET 1: Optimizing Matrix Multiplication & CUDA

...

Try not to add additional cells nor rearrange any that exist now as it will mess with our autograder (but feel free to open another Colab/notebook on the side).  

**Note:** the autograder on Gradescope might take a while (~5+ min) to run (the benchmarking exercises at the end slow things down, feel free to comment them all out).  The CUDA exercises will be graded run manually.

### Make sure to submit your final notebook with all of your solutions to Gradescope!
[Direct Gradescope Link](https://www.gradescope.com/courses/820552)

### Before Starting

This problem set uses NVCC (NVIDIA CUDA Compiler) within Google Colab to compile and run CUDA code. Google Colab provides a convenient (and free!) environment with GPU support for executing these programs.  At the top right corner, under the arrow dropdown next to the `Connect` button, select "Change runtime type" and choose `T4 GPU` for this problem set.  Note, across your account, you can only have one runtime connected to a GPU at a given time.

We're in an interactive _Python_ notebook.  So, of course, running C++ code is not going to be as easy as it is to run Python code.  That said, we'll once again use helpers to make this cleaner.

Our plugins `%%cpurun` and `%%gpurun` save, compile, and run your C++ and/or CUDA code in the cell.  This is different than before where the helper simply *saved* the file, and then we would compile using `g++` and run it ourselves.  But now, since you remember the nuances of working with a compiled language, we'll just bundle it all up.

In [ ]:
# make sure CUDA is installed
!nvcc --version

# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

# Install some magic to run and save .cpp programs
!curl -o ./cpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/cpu_runner.py
%load_ext cpu_runner

# Install some magic to run and save .cu C++ CUDA programs
!curl -o ./gpu_runner.py https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/gpu_runner.py
%load_ext gpu_runner

# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

One other note: we import a few `.o` "dot-oh" object and `.h` header files to bring along some printing utilities and fuzz-testing in an obscured way (we wouldn't want to give away the solutions 😛).

As you step away and return to this problem set, you will often find that Colab has disconnected your runtime and deleted any files.  You'll need to re-run cells like the above which import the magic compilation utilities as well as the cells with `curl` imports of the various `.o` and `.h` files.

## Problem 1: Python Matrix Multiply (again)
In the last problem set, you explored how to write a simple matrix multiplication in Python using a naïve implementation for representing matrices using lists of lists.

In `C++` there are ways to mimic this style:
- multi-dimensional arrays like `int my_matrix[N][M]`
- `std::vector<std::vector<float> >`

But it gets quite a bit more difficult when we try to do this using CUDA.  An easier way to handle such a problem is to put everything into one contiguous chunk of memory in an organized way.  As we've discussed in class, there are two standard methods for representing/organizing our matrix in memory:
1. row-major order
2. column-major order

These are visualized below:
![row-major](https://raw.githubusercontent.com/COMS-BC3159-F24/image_assets/main/row-col-major.png)

In the written part of PS0, you reacquainted yourself with the standard matrix multiplication algorithm:
$$C_{ij} = \sum_{k=1}^{n} A_{ik} \times B_{kj}$$

Then, in the coding part of PS0, using `list`s of `list`s, you computed the product of a matrix $A$ (in gray) which is $4\times5$ and $B$ (blue) of dimensions $5\times4$ whose product results in $C$ (purple) of size $4\times4$.

![matrix mult example](https://raw.githubusercontent.com/COMS-BC3159-F24/image_assets/main/matmul_example.png)

This time, write a matrix multiplication implementation that indexes into provided **row-major order** matrices to compute their product.  As before, we've provided the matrices (note: they're _not_ `list`s of `list`s this time; instead, just one flat `list`) and some printing functions (albeit a bit obscured to leave the challenge of figuring out the indexing for the actual multiplication up to you)!

In [ ]:
# Helper function to print a matrix represented as a flat list in a row-major order
def printFlatMatrix(matrix, num_rows, num_cols, width=6):
    for i in range(num_rows):
        formatted_row = ''.join(f'{matrix[j]:{width}}'
                                for j in range(
                                    i * num_cols, i * num_cols + num_cols))
        print(formatted_row)

# Matrices' dimensions (can't be inferred from list len)
rows_A, cols_A = 4, 5
rows_B, cols_B = 5, 4

# Note: single list in row-major
A = [
    0, 1, 2, 3, 4,
    5, 6, 7, 8, 9,
    10, 11, 12, 13, 14,
    15, 16, 17, 18, 19
]
B = [
    20, 21, 22, 23,
    24, 25, 26, 27,
    28, 29, 30, 31,
    32, 33, 34, 35,
    36, 37, 38, 39
]
C = [0] * (rows_A * cols_B)

In [ ]:
def matrixMultiply(A, B, rows_A, cols_B):
    ...
    return C

C = matrixMultiply(A, B, rows_A, cols_B)
printFlatMatrix(C, rows_A, cols_B)

## C++ Malloc Practice [Ungraded]
As a warmup, let's review how we allocate memory dynamically.  Allocate memory for a row of 5 integers and fill the row with values 0-4.  Print the array, and don't forget to `free()` when you're done!

In [ ]:
%%cpurun -n malloc_example.cpp
#include <cstdio>  // For printf
#include <cstdlib> // For malloc and free

int main() {
    // Allocate memory for a row of 5 integers
    int* rowA = nullptr; // TODO: Actually allocate!
    if (rowA == nullptr) {
        std::fprintf(stderr, "Memory allocation failed\n");
        return 1;  // Exit with an error code
    }

    // Fill the array with values
    for (int i = 0; i < 5; i++) {
        rowA[i] = i;
    }

    // Alternative way to fill the array
    int* temp = rowA;
    for (int i = 0; i < 5; i++) {
        *temp = i;
        temp++; // Pointer arithmetic, increments temp pointer raw value by sizeof(int) each time!
    }

    // Print each element of the array using printf
    printf("Elements of rowA:\n");
    for (int i = 0; i < 5; i++) {
        printf("%d ", rowA[i]);
    }
    printf("\n");

    // Free allocated memory
    free(rowA);

    return 0;
}


## Problem 2: C++ Matrix Multiply

Before we jump to a CUDA implementation, let's continue warming up the background necessary by writing a `C++` implementation of matrix multiplication.

Although this could be done easily using `std::vector`s of `std::vector`s or multi-dimensional arrays, let's practice allocating just **one** contiguous chunk of memory per each of our matrices and use row-major ordering to organize each matrix in memory.

We've provided a `print_int_matrix()` function in the `matrix_lib.h` header.  It assumes row-major order in one contiguous chunk of memory as described above.  The function signature looks as follows:
```c
void print_int_matrix(const int* matrix, int num_rows, int num_cols)
```

We link the provided `matrix_lib.o` file via the magic `%%cpurun` compilation command. (It's obscured in this way so that we don't take the fun out of figuring out the indexing for your matrices!)

This time all the fun is left to you!  
1. Allocate the chunks of memory for $A$, $B$, and $C$.
2. Fill in $A$ and $B$ matrices as shown in the image above and the Python exercise in Problem 1.
3. Write the multiply!  (And print only $C$ via the `print_int_matrix()` function provided.)

Your output should look exactly as follows:
```text
  320   330   340   350
 1020  1055  1090  1125
 1720  1780  1840  1900
 2420  2505  2590  2675
```


In [ ]:
!curl -o ./matrix_lib.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.o
!curl -o ./matrix_lib.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.h

In [ ]:
%%cpurun -n cpp_int_matmul.cpp -o matrix_lib.o
#include <cstdio>
#include <cstdlib>
#include "matrix_lib.h"

// TODO: Write your matrix multiplication
//       where A: MxN, B: NxP, C: MxP
//       and you write C in place
void matrixMultiplyCPU_int(int* A, int* B, int* C, int M, int N, int P) {
...
...
...
...
...
...
...
...
}



int main() {
    const int rows_A = 4, cols_A = 5;
    const int rows_B = 5, cols_B = 4;
    const int rows_C = rows_A, cols_C = cols_B;

    // Allocate matrix A
    int* A = nullptr; // TODO!
    ...
    if (A == nullptr) {
        fprintf(stderr, "Memory allocation for A failed\n");
        return 1;
    }

    // Fill matrix A (use print function to temporarily help initialize)
    ...
    ...
    ...
    ...
    ...

    // Allocate matrix B
    int* B = nullptr; // TODO!
    ...
    if (B == nullptr) {
        fprintf(stderr, "Memory allocation for B failed\n");
        ...
        return 1;
    }

    // Fill matrix B
    ...
    ...
    ...
    ...
    ...

    // Allocate matrix C (result matrix)
    int* C = nullptr; // TODO!
    ...
    if (C == nullptr) {
        fprintf(stderr, "Memory allocation for C failed\n");
        ...
        ...
        return 1;
    }

    // Perform matrix multiplication
    matrixMultiplyCPU_int(A, B, C, rows_A, cols_A, cols_B);

    // Print result
    print_int_matrix(C, rows_A, cols_B);

    // Free allocated memory
    ...
    ...
    ...

    return 0;
}

## Problem 3: Float Matrix Multiply in C++

Before moving forward, let's run this exercise again, but using `float` instead of `int`.  In the same header, we've provided a `print_float_matrix()` for you to use (which prints 2 decimals of precision).  Your output should look exactly as follows:
```text
 320.00  330.00  340.00  350.00
1020.00 1055.00 1090.00 1125.00
1720.00 1780.00 1840.00 1900.00
2420.00 2505.00 2590.00 2675.00
```

In [ ]:
!curl -o ./matrix_lib.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.o
!curl -o ./matrix_lib.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.h

In [ ]:
%%cpurun -n cpp_float_matmul.cpp -o matrix_lib.o
#include <cstdio>
#include <cstdlib>
#include "matrix_lib.h"

// TODO: Write your matrix multiplication
//       where A: MxN, B: NxP, C: MxP
//       and you write C in place
void matrixMultiplyCPU_float(float* A, float* B, float* C, int M, int N, int P) {
...
...
...
...
...
...
...
...
}



int main() {
    const int rows_A = 4, cols_A = 5;
    const int rows_B = 5, cols_B = 4;
    const int rows_C = rows_A, cols_C = cols_B;

    // Allocate matrix A
    float* A = nullptr; // TODO!
    ...
    if (A == nullptr) {
        fprintf(stderr, "Memory allocation for A failed\n");
        return 1;
    }

    // Fill matrix A (use print function to temporarily help initialize)
    ...
    ...
    ...
    ...
    ...

    // Allocate matrix B
    float* B = nullptr; // TODO!
    ...
    if (B == nullptr) {
        fprintf(stderr, "Memory allocation for B failed\n");
        ...
        return 1;
    }

    // Fill matrix B
    ...
    ...
    ...
    ...
    ...

    // Allocate matrix C (result matrix)
    float* C = nullptr; // TODO!
    ...
    if (C == nullptr) {
        fprintf(stderr, "Memory allocation for C failed\n");
        ...
        ...
        return 1;
    }

    // Perform matrix multiplication
    matrixMultiplyCPU_float(A, B, C, rows_A, cols_A, cols_B);

    // Print result
    print_float_matrix(C, rows_A, cols_B);

    // Free allocated memory
    ...
    ...
    ...

    return 0;
}

Let's put it to the test!  Beyond our simple example, we provide a fuzz test (against our reference `C++` implementation) to help you validate your implementation is working as expected!  All you need to do is paste in your `matrixMultiplyCPU_float()` implementation and run!

First, we'll import our object/header.

In [ ]:
!curl -o ./cpu_fuzz_test.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/cpu_fuzz_test.o
!curl -o ./cpu_fuzz_test.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/cpu_fuzz_test.h

In [ ]:
%%cpurun -n student_testing.cpp -o matrix_lib.o,cpu_fuzz_test.o
#include "matrix_lib.h"
#include "cpu_fuzz_test.h"

// Student implementation of CUDA matrix multiplication
void matrixMultiplyCPU_float(float* A, float* B, float* C, int M, int N, int P) {
    ...
}

int main() {
    fuzzTestMatrixMultiplication(10, 50);  // Run fuzz tests
    return 0;
}


## CUDA Compiling Basics [Ungraded]
Print the following by writing a `__global__` kernel and invoking it.
```text
Hello from the GPU!
```
We'll add a `cudaDeviceSynchronize` at the end of your `main()` function to wrap up neatly!  Recall what happens if you don't include that line...(and if you don't remember, then try commenting it out)!

#### Not seeing any output?  Did you remember to change your runtime?
Don't forget to change your runtime in Google Colab to a T4 GPU.

![change runtime](https://raw.githubusercontent.com/COMS-BC3159-F24/image_assets/main/changeruntime.png)

In [ ]:
%%gpurun -n my_kernel.cu
#include <cstdio>
#include <cuda_runtime.h>

// This function runs on the GPU
__global__ void helloWorldKernel() {
    printf("Hello World!\n");
}

// This function runs on the CPU
__host__
int main() {
    // Launch the kernel on the GPU
    helloWorldKernel<<<1, 1>>>();

    // Wait for the kernel to finish executing
    cudaDeviceSynchronize();

    return 0;
}

## Problem 4: CUDA Matrix Multiply
The time has finally arrived...to "CUDA-fy" your matrix multiplication code.  You've practiced examples in class and in our HelloCUDA exercises.  Now, let's do the necessary allocations, write a kernel, move and copy data, and run!

To constrain your solution, let's use a 2D block of dimensions 2x2.  Yes, this is tiny.  Something more standard like 32 to collect the threads in a warp is typically sensible for larger matrices.  But we have a small output matrix, and this is a first exercise, so let's keep things simple for now.  (As a follow-on exercise, you could try tiling your matrix and finding ways to leverage shared memory in smarter ways.)

You'll need to make sure your grid and respective grid dimensions support the whole matrix.  Your kernel should compute **one** element of the resulting $C$ matrix.  (We have so much compute available, so let's go parallel heavy!)

Your code from the `C++` "CPU" version of matrix multiply in Problem 3 can be used as a starting point here.  In fact, to make checking things easy, we have provided a framework of a script that leverages your `matrixMultiplyCPU_float()` from Problem 3 (just paste it here) for a reference comparison.

Follow each step provided in the "comments" `// TODO`, etc. and we've already included the print statements (using our helpers from `#include "matrix_lib.h"` again) that should provide the exact output we expect:

```text
CPU Matrix Multiply:
 320.00  330.00  340.00  350.00
1020.00 1055.00 1090.00 1125.00
1720.00 1780.00 1840.00 1900.00
2420.00 2505.00 2590.00 2675.00

GPU Matrix Multiply:
 320.00  330.00  340.00  350.00
1020.00 1055.00 1090.00 1125.00
1720.00 1780.00 1840.00 1900.00
2420.00 2505.00 2590.00 2675.00

Matrices match!
```

In [ ]:
!curl -o ./matrix_lib.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.o
!curl -o ./matrix_lib.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.h

In [ ]:
%%gpurun -n cuda_matmul.cu -o matrix_lib.o
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>
#include "matrix_lib.h"

// Paste your Problem 3 solution for comparison
void matrixMultiplyCPU_float(float* A, float* B, float* C, int M, int N, int P) {
...
...
...
...
...
...
...
...
}

// CUDA kernel to multiply two matrices A[M][N] * B[N][P] = C[M][P]
// Specify what work to complete via threadIdx, blockIdx, blockDim
// Compute one element of the resulting matrix
__global__ void matrixMultiplyCUDA(float* A, float* B, float* C, int M, int N, int P) {
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
}

// Compare matrices represented as flat arrays with single loop
// Allow for some floating-point error (GPU and CPU may differ slightly)
bool compareMatrices(const float* A, const float* B, int total_elements) {
    float epsilon = 0.001f; // Tolerance for floating-point comparison
    int precision_issues = 0;

    // Note: not traversing in row-major order
    for (int i = 0; i < total_elements; ++i) {
        float diff = fabs(A[i] - B[i]);
        if (diff > epsilon) {
            printf("Matrices differ at element %d: A[%d] = %f, B[%d] = %f\n", i, i, A[i], i, B[i]);
            return false;
        } else if (diff > epsilon / 10 && diff <= epsilon) {
            precision_issues++;
        }
    }

    if (precision_issues > 0) {
        fprintf(stderr, "Warning: %d elements had values close to the precision threshold.\n", precision_issues);
    }

    return true;
}

int main() {
    const int rows_A = 4, cols_A = 5;
    const int rows_B = 5, cols_B = 4;
    const int rows_C = rows_A, cols_C = cols_B;

    // Allocate memory for matrices A, B, C
    // * C_gpu will be for copying and comparison purposes
    float* h_A = nullptr; // TODO
    ...
    float* h_B = nullptr; // TODO
    ...
    float* h_C_cpu = nullptr; // TODO
    ...
    float* h_C_gpu = nullptr;
    ...

    // Fill matrix A and B (similar to as you did in the CPU version)
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...

    // Perform matrix multiplication on CPU (C++)
    matrixMultiplyCPU_float(h_A, h_B, h_C_cpu, rows_A, cols_A, cols_B);
    printf("CPU Matrix Multiply:\n");
    print_float_matrix(h_C_cpu, rows_C, cols_C);

    // Perform matrix multiplication on GPU (CUDA)
    // create d_ device pointers, allocate GPU memory, and fill the memory
    float *d_A, *d_B, *d_C;
    ...
    ...
    ...
    ...
    ...

    // Define grid/block dimensions, then launch the kernel
    ...
    ...
    ...

    // Copy result back to host (h_C_gpu) for comparison
    ...

    printf("\nGPU Matrix Multiply:\n");
    print_float_matrix(h_C_gpu, rows_C, cols_B);

    // Compare the CPU and GPU results
    if (compareMatrices(h_C_cpu, h_C_gpu, rows_C*cols_C)) {
        printf("\nMatrices match!\n");
    } else {
        printf("\nMatrices do NOT match.\n");
    }

    // Free memory
    ...
    ...
    ...
    ...
    ...
    ...
    ...

    return 0;
}


Let's put it to the test!  Beyond our simple example, we provide a fuzz test (against our reference `C++` implementation) to help you validate your implementation is working as expected!  All you need to do is paste in your CUDA kernel and run!

In [ ]:
!curl -o ./fuzz_test.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/fuzz_test.o
!curl -o ./fuzz_test.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/fuzz_test.h

In [ ]:
%%gpurun -n student_cuda_testing.cu -o matrix_lib.o,fuzz_test.o
#include "matrix_lib.h"
#include "fuzz_test.h"

// Your CUDA matrix multiplication kernel (one element of C per thread)
__global__ void matrixMultiplyCUDA(float* A, float* B, float* C, int M, int N, int P) {
    ...
}

int main() {
    fuzzTestMatrixMultiplication(10, 50);  // Run fuzz tests
    cudaDeviceSynchronize();
    return 0;
}


## Problem 5: One More Time!  (CUDA Large)
Now, let's do another matrix multiplication leveraging much of the same code you used before.  This time, we'll use a 2D block of dimensions 32x32.  You'll need to make sure your grid dimensions use enough blocks to cover the entire matrix.

Again, your kernel should compute **one** element of the resulting $C$ matrix via each thread.

Your code from the `C++` "CPU" version of matrix multiply in Problem 3 will be used as reference here, again.  We have provided the same framework of a script that leverages your `matrixMultiplyCPU_float()` from Problem 3 (just paste it here) for a reference comparison.

This time, we provide the functions to "fill in" your matrices with a specified pattern of 1s and 0s.

Follow each step provided in the "comments" `// TODO`, etc. and we've already included the print statements (using our helpers from `#include "matrix_lib.h"` again) that should provide the exact output we expect:

```text
CPU Matrix Multiply:
1000.00    0.00 1000.00    0.00
   0.00    0.00    0.00    0.00
...

GPU Matrix Multiply:
1000.00    0.00 1000.00    0.00
   0.00    0.00    0.00    0.00
...

Matrices match!
```

In [ ]:
!curl -o ./matrix_lib.o https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.o
!curl -o ./matrix_lib.h https://raw.githubusercontent.com/COMS-BC3159-F24/helpers/main/matrix_lib.h

In [ ]:
%%gpurun -n big_cuda.cu -o matrix_lib.o
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>
#include "matrix_lib.h"

// Paste your Problem 3 solution for comparison
void matrixMultiplyCPU_float(float* A, float* B, float* C, int M, int N, int P) {
...
...
...
...
...
...
...
...
}

// CUDA kernel to multiply two matrices A[M][N] * B[N][P] = C[M][P]
// Specify what work to complete via threadIdx, blockIdx, blockDim
// Compute one element of the resulting matrix
__global__ void matrixMultiplyCUDA(float* A, float* B, float* C, int M, int N, int P) {
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
    ...
}

// Compare matrices represented as flat arrays with single loop
// Allow for some floating-point error (GPU and CPU may differ slightly)
bool compareMatrices(const float* A, const float* B, int total_elements) {
    float epsilon = 0.001f; // Tolerance for floating-point comparison
    int precision_issues = 0;

    // Note: not traversing in row-major order
    for (int i = 0; i < total_elements; ++i) {
        float diff = fabs(A[i] - B[i]);
        if (diff > epsilon) {
            printf("Matrices differ at element %d: A[%d] = %f, B[%d] = %f\n", i, i, A[i], i, B[i]);
            return false;
        } else if (diff > epsilon / 10 && diff <= epsilon) {
            precision_issues++;
        }
    }

    if (precision_issues > 0) {
        fprintf(stderr, "Warning: %d elements had values close to the precision threshold.\n", precision_issues);
    }

    return true;
}

int main() {
    const int rows_A = 2000, cols_A = 2000;
    const int rows_B = 2000, cols_B = 2000;
    const int rows_C = rows_A, cols_C = cols_B;

    // Allocate memory for matrices A, B, C
    // * C_gpu will be for copying and comparison purposes
    float* h_A = nullptr; // TODO
    ...
    float* h_B = nullptr; // TODO
    ...
    float* h_C_cpu = nullptr; // TODO
    ...
    float* h_C_gpu = nullptr;
    ...

    // Fill matrix A with alternating 0 and 1 in a checkerboard pattern
    for (int idx = 0; idx < rows_A * cols_A; ++idx) {
        h_A[idx] = ((idx / cols_A + idx % cols_A) % 2 == 0) ? 1.0f : 0.0f;  // Checkerboard pattern
    }

    // Fill matrix B with alternating 0 and 1 in a different pattern
    for (int idx = 0; idx < rows_B * cols_B; ++idx) {
        h_B[idx] = (((idx / cols_B) % 2 == 0) && ((idx % cols_B) % 2 == 0)) ? 1.0f : 0.0f;
    }

    // Perform matrix multiplication on CPU (C++)
    matrixMultiplyCPU_float(h_A, h_B, h_C_cpu, rows_A, cols_A, cols_B);
    printf("CPU Matrix Multiply:\n");
    print_float_matrix(h_C_cpu, rows_C, cols_C);

    // Perform matrix multiplication on GPU (CUDA)
    // create d_ device pointers, allocate GPU memory, and fill the memory
    float *d_A, *d_B, *d_C;
    ...
    ...
    ...
    ...
    ...

    // Define grid/block dimensions, then launch the kernel
    ...
    ...
    ...

    // Copy result back to host (h_C_gpu) for comparison
    ...

    printf("\nGPU Matrix Multiply:\n");
    print_float_matrix(h_C_gpu, rows_C, cols_B);

    // Compare the CPU and GPU results
    if (compareMatrices(h_C_cpu, h_C_gpu, rows_C*cols_C)) {
        printf("\nMatrices match!\n");
    } else {
        printf("\nMatrices do NOT match.\n");
    }

    // Free memory
    ...
    ...
    ...
    ...
    ...
    ...
    ...

    return 0;
}


---

## Benchmarking! [Ungraded]
Nice!  Now that you're done with the "work" for this PSET, we want to start adding some runtime numbers to this whole "optimization" concept.

In class, and more widely, we've discussed the slowness of Python *v.* C (an interpreted versus compiled language).  We've talked about the systems effects of cache and access patterns (as simple as loop orders).  And we've discussed the potential speedup offered by parallelism with the caveat that systems and memory factors may impact _more_ than parallelism can help.

Let's run some trials now to see these effects in reality!  We'll use $N \times N$ matrices where $N=2000$.

Read each cell, consider the accuracy of the timestamps, and then run (and wait...)

### Python (naïve _v._ Numpy)
This may run for a while...

In [ ]:
import numpy as np
import time

def matrix_multiply(A, B):
    N = len(A)
    C = [[0] * N for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i][j] += A[i][k] * B[k][j]
    return C

def benchmark_matmul(N, use_numpy=False):
    # Initialize matrices with random values
    if use_numpy:
        A = np.random.randint(0, 10, (N, N))
        B = np.random.randint(0, 10, (N, N))
    else:
        A = [[np.random.randint(0, 10) for _ in range(N)] for _ in range(N)]
        B = [[np.random.randint(0, 10) for _ in range(N)] for _ in range(N)]

    # Benchmark
    start_time = time.time()

    if use_numpy:
        C = np.dot(A, B)
    else:
        C = matrix_multiply(A, B)

    end_time = time.time()

    print(f"Time taken for matrix multiplication of size {N}x{N}: {end_time - start_time:.6f} seconds (using {'NumPy' if use_numpy else 'custom function'})")

if __name__ == "__main__":
    N = 2000  # Example matrix size
    print("\nBenchmarking NumPy matrix multiplication:")
    benchmark_matmul(N, use_numpy=True)

    print("\nBenchmarking custom matrix multiplication:")
    # NOTE: only uncomment this for one test run (do not submit with this uncommented)
    # benchmark_matmul(N)


### C++
You'll note that the naïve C++ implementation may not necessarily beat Numpy, either!  Their optimizations (BLAS, etc.) really are excellent!

In [ ]:
%%cpurun -n benchSimpleMatrixMul.cpp

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <chrono>

// Define matrix size N
const int N = 2000;

// Static global arrays
int A[N][N];
int B[N][N];
int C[N][N];

void matrix_multiply() {
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            C[i][j] = 0;
            for (int k = 0; k < N; ++k) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }
}

void benchmark_matmul() {
    // Initialize matrices with random values
    srand(static_cast<unsigned>(time(0)));
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            A[i][j] = rand() % 10;
            B[i][j] = rand() % 10;
        }
    }

    // Benchmark
    auto start = std::chrono::high_resolution_clock::now();
    matrix_multiply();
    auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> duration = end - start;
    std::cout << "Time taken for matrix multiplication of size " << N << "x" << N << ": "
         << duration.count() << " seconds" << std::endl;
}

int main() {
    benchmark_matmul();
    return 0;
}

### Compiler Optimizations

This implementation outperforms both naïve C++ and NumPy by leveraging advanced compiler optimizations such as `-Ofast`, `-funroll-loops`, and `-march=native`. These optimizations enhance performance by exploiting architecture-specific details!  Many people even [walk](https://arxiv.org/pdf/2105.07202) the large search space of optimization flags to find the best configuration.

In [ ]:
!cp benchSimpleMatrixMul.cpp benchmarkOptimizedMatrixMul.cpp
!g++ -Ofast -funroll-loops -march=native benchmarkOptimizedMatrixMul.cpp -o benchmarkOptimizedMatrixMul
!./benchmarkOptimizedMatrixMul

### Cache-Friendlier Implementations
In class, we showed that `averageMatrix()` where loop order changed could be 5x faster.  Before you dig into the differences in the code below, could you think of optimizations for the multiplication given how memory is laid out for C++?  Also consider caching and ways to share temporally local data.

![matrix mult example](https://raw.githubusercontent.com/COMS-BC3159-F24/image_assets/main/rowmajor.png)

In [ ]:
%%cpurun -n cacheFriendlyMatrixMul.cpp

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <chrono>

// Define matrix size N
const int N = 2000;

// Static global arrays
int A[N][N];
int B[N][N];
int C[N][N];

void matrix_multiply() {
    for (int i = 0; i < N; ++i) {
        for (int k = 0; k < N; ++k) {
            for (int j = 0; j < N; ++j) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }
}


void benchmark_matmul() {
    // Initialize matrices with random values
    srand(static_cast<unsigned>(time(0)));
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            A[i][j] = rand() % 10;
            B[i][j] = rand() % 10;
            C[i][j] = 0;
        }
    }

    // Benchmark
    auto start = std::chrono::high_resolution_clock::now();
    matrix_multiply();
    auto end = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> duration = end - start;
    std::cout << "Time taken for matrix multiplication of size " << N << "x" << N << ": "
         << duration.count() << " seconds" << std::endl;
}

int main() {
    benchmark_matmul();
    return 0;
}

Adding optimization flags, now:

In [ ]:
!cp cacheFriendlyMatrixMul.cpp cacheFriendlyMatrixMulOpt.cpp
!g++ -Ofast -funroll-loops -march=native cacheFriendlyMatrixMulOpt.cpp -o cacheFriendlyMatrixMulOpt
!./cacheFriendlyMatrixMulOpt

**WOW!**  Now we're really flying?!  If you have extra time, take this a step further!  
1. Can you write a program that's even more cache aware?
2. What about benchmarking your CUDA implementation?  Is it better/worse?  Why so?